<a href="https://colab.research.google.com/github/davidclizbe/datascience/blob/master/ClizbeOverfittingAndRegularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load the houseprices data from Thinkful's database.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

houseprices = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [ ]:
houseprices

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,...,centralair,electrical,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,...,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,None,None,None,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,...,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,None,MnPrv,None,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,...,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,None,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,

Reimplement your model from the previous checkpoint.

In [ ]:
housedata = pd.get_dummies(houseprices, dummy_na=True)
housedata.head()

,id,mssubclass,lotfrontage,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,totrmsabvgrd,fireplaces,garageyrblt,garagecars,garagearea,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,mszoning_C (all),mszoning_FV,...,garagecond_Fa,garagecond_Gd,garagecond_Po,garagecond_TA,garagecond_nan,paveddrive_N,paveddrive_P,paveddrive_Y,paveddrive_nan,poolqc_Ex,poolqc_Fa,poolqc_Gd,poolqc_nan,fence_GdPrv,fence_GdWo,fence_MnPrv,fence_MnWw,fence_nan,miscfeature_Gar2,miscfeature_Othr,miscfeature_Shed,miscfeature_TenC,miscfeature_nan,saletype_COD,saletype_CWD,saletype_Con,saletype_ConLD,saletype_ConLI,saletype_ConLw,saletype_New,saletype_Oth,saletype_WD,saletype_nan,salecondition_Abnorml,salecondition_AdjLand,salecondition_Alloca,salecondition_Family,salecondition_Normal,salecondition_Partial,salecondition_nan
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,208500,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,181500,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,223500,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,140000,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,250000,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [ ]:
housedata.dropna(inplace=True)

In [ ]:
y = housedata['saleprice']

X = housedata.drop('saleprice', axis=1)
X.head()

,id,mssubclass,lotfrontage,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,totrmsabvgrd,fireplaces,garageyrblt,garagecars,garagearea,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,mszoning_C (all),mszoning_FV,mszoning_RH,...,garagecond_Fa,garagecond_Gd,garagecond_Po,garagecond_TA,garagecond_nan,paveddrive_N,paveddrive_P,paveddrive_Y,paveddrive_nan,poolqc_Ex,poolqc_Fa,poolqc_Gd,poolqc_nan,fence_GdPrv,fence_GdWo,fence_MnPrv,fence_MnWw,fence_nan,miscfeature_Gar2,miscfeature_Othr,miscfeature_Shed,miscfeature_TenC,miscfeature_nan,saletype_COD,saletype_CWD,saletype_Con,saletype_ConLD,saletype_ConLI,saletype_ConLw,saletype_New,saletype_Oth,saletype_WD,saletype_nan,salecondition_Abnorml,salecondition_AdjLand,salecondition_Alloca,salecondition_Family,salecondition_Normal,salecondition_Partial,salecondition_nan
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [ ]:
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: saleprice, dtype: int64

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

,id,mssubclass,lotfrontage,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,totrmsabvgrd,fireplaces,garageyrblt,garagecars,garagearea,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,mszoning_C (all),mszoning_FV,mszoning_RH,...,garagecond_Fa,garagecond_Gd,garagecond_Po,garagecond_TA,garagecond_nan,paveddrive_N,paveddrive_P,paveddrive_Y,paveddrive_nan,poolqc_Ex,poolqc_Fa,poolqc_Gd,poolqc_nan,fence_GdPrv,fence_GdWo,fence_MnPrv,fence_MnWw,fence_nan,miscfeature_Gar2,miscfeature_Othr,miscfeature_Shed,miscfeature_TenC,miscfeature_nan,saletype_COD,saletype_CWD,saletype_Con,saletype_ConLD,saletype_ConLI,saletype_ConLw,saletype_New,saletype_Oth,saletype_WD,saletype_nan,salecondition_Abnorml,salecondition_AdjLand,salecondition_Alloca,salecondition_Family,salecondition_Normal,salecondition_Partial,salecondition_nan
486,486,20,80.0,9600,5,7,1950,2007,0.0,607,0,506,1113,1113,0,0,1113,0,0,1,0,3,1,5,1,1950.0,1,264,0,80,120,0,0,0,0,7,2009,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
30,31,70,50.0,8500,4,4,1920,1950,0.0,0,0,649,649,649,668,0,1317,0,0,1,0,3,1,6,0,1920.0,1,250,0,54,172,0,0,0,0,7,2008,1,0,0,...,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1282,1283,20,61.0,8800,5,7,1977,2008,0.0,532,144,364,1040,1040,0,0,1040,0,0,2,0,3,1,5,0,1977.0,2,484,0,0,0,0,288,0,0,9,2009,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1119,1120,20,70.0,7560,5,5,1959,1959,0.0,369,0,671,1040,1040,0,0,1040,0,0,1,0,3,1,6,0,1959.0,1,286,140,0,252,0,0,0,0,7,2006,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
501,501,160,21.0,1890,6,5,1973,1973,285.0,356,0,316,672,672,546,0,1218,0,0,1,1,3,1,7,0,1973.0,1,264,144,28,0,0,0,0,0,5,2007,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [ ]:
import statsmodels.api as sm
import numpy as np

model = sm.OLS(y_train,X_train)
results = model.fit()
results.params

id                          -1.350840
mssubclass                 125.334242
lotfrontage                 68.113344
lotarea                      1.168375
overallqual               5870.778970
                             ...     
salecondition_Alloca    -85386.009283
salecondition_Family    -93658.623186
salecondition_Normal    -89591.452284
salecondition_Partial   -72455.604080
salecondition_nan            0.000000
Length: 332, dtype: float64

In [ ]:
results.tvalues

id                      -0.713929
mssubclass               0.988926
lotfrontage              1.209215
lotarea                  7.561889
overallqual              4.506304
                           ...   
salecondition_Alloca    -3.461804
salecondition_Family    -4.073090
salecondition_Normal    -3.942225
salecondition_Partial   -2.640843
salecondition_nan             NaN
Length: 332, dtype: float64

In [ ]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.952
Model:                            OLS   Adj. R-squared:                  0.935
Method:                 Least Squares   F-statistic:                     55.66
Date:                Tue, 23 Jun 2020   Prob (F-statistic):               0.00
Time:                        19:30:27   Log-Likelihood:                -10028.
No. Observations:                 896   AIC:                         2.053e+04
Df Residuals:                     661   BIC:                         2.165e+04
Df Model:                         234                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
id                       -1.3508      1.892     -0.714      0.476      -5.066       2.364
mssubclass              125.3342    126.738      0.989      0.323    -123.523     374.191
lotfrontage              68.1133     56.329      1.209      0.227     -42.491     178.718
lotarea                   1.1684      0.155      7.562      0.000       0.865       1.472
overallqual            5870.7790   1302.793      4.506      0.000    3312.668    8428.890
overallcond            5990.4870   1170.228      5.119      0.000    3692.675    8288.299
yearbuilt               335.3575    105.843      3.168      0.002     127.529     543.186
yearremodadd             59.3645     72.378      0.820      0.412     -82.755     201.484
masvnrarea               17.0169      6.704      2.538      0.011       3.853      30.181
bsmtfinsf1               17.1094      3.669      4.663      0.000       9.905      24.314
bsmtfinsf2                7.7231      7.568      1.020      0.308      -7.137      22.583
bsmtunfsf                 0.5158      3.588      0.144      0.886      -6.530       7.562
totalbsmtsf              25.3484      5.218      4.857      0.000      15.102      35.595
firstflrsf                3.2151      9.593      0.335      0.738     -15.621      22.051
secondflrsf              29.4742      8.064      3.655      0.000      13.640      45.308
lowqualfinsf             11.3764     22.679      0.502      0.616     -33.154      55.907
grlivarea                44.0656      8.817      4.998      0.000      26.752      61.379
bsmtfullbath           3159.3680   2412.593      1.310      0.191   -1577.901    7896.637
bsmthalfbath           1597.2847   3732.896      0.428      0.669   -5732.478    8927.048
fullbath               4729.0170   2850.425      1.659      0.098    -867.962    1.03e+04
halfbath               1389.4973   2602.806      0.534      0.594   -3721.267    6500.262
bedroomabvgr          -6047.8317   1813.701     -3.335      0.001   -9609.142   -2486.522
kitchenabvgr          -1.629e+04   8158.008     -1.997      0.046   -3.23e+04    -273.103
totrmsabvgrd           1192.5584   1203.360      0.991      0.322   -1170.310    3555.427
fireplaces             2850.2961   3527.379      0.808      0.419   -4075.921    9776.513
garageyrblt               0.9574     75.335      0.013      0.990    -146.967     148.882
garagecars             3576.7538   2750.326      1.300      0.194   -1823.674    8977.182
garagearea               10.1921      9.734      1.047      0.295      -8.921      29.305
wooddecksf               20.4847      7.436      2.755      0.006       5.884      35.085
openporchsf              16.9788     14.865      1.142      0.254     -12.209      46.167
enclosedporch            17.7551     16.389      1.083      0.279     -14.425      49.935
threessnporch          

In [ ]:
results.summary

<bound method RegressionResults.summary of <statsmodels.regression.linear_model.OLSResults object at 0x7fbdf7179c18>>

Try OLS, Lasso, Ridge, and ElasticNet regression using the same model specification. This time, you need to do k-fold cross-validation to choose the best hyperparameter values for your models. Scikit-learn has RidgeCV, LassoCV, and ElasticNetCV that you can utilize to do this. Which model is the best? Why?

In [ ]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV


lcv = LassoCV(cv= 5)

model = lcv.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
model.score(X_train, y_train)

0.7050033828578876

In [ ]:
model.score(X_test, y_test)

0.7494347776591215

In [ ]:
rdg = RidgeCV(cv = 5)
model = rdg.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
model.score(X_train, y_train)

0.9292372107693555

In [ ]:
model.score(X_test, y_test)

0.8631590030288905

In [ ]:
elst = ElasticNetCV(cv = 5)
model = elst.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
model.score(X_train, y_train)

0.5593971675093017

In [ ]:
model.score(X_test, y_test)

0.6063107266829578